In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

import gensim
import gensim.corpora as corpora
from gensim.utils import  simple_preprocess
from gensim.models import CoherenceModel

import spacy

import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
# import dataset
df = pd.read_csv('cleaned_data.csv')
df.head()

,Consumer complaint narrative,Issue,Product
0,My minimum payment for my Fortiva Retail Credi...,Problem when making payments,Credit card or prepaid card
1,I HAD MY IDENTITY STOLEN AND SOMEONE CREATED F...,Problem with a credit reporting company's inve...,"Credit reporting, credit repair services, or o..."
2,I'm sending this complaint to inform AGAIN the...,Incorrect information on your report,"Credit reporting, credit repair services, or o..."
3,"IDENTITY THEFT XX/XX/XXXX XXXX XXXX XXXX, XXXX...",Incorrect information on your report,"Credit reporting, credit repair services, or o..."
4,I have filed numerous complaints with the 3 cr...,Problem with a credit reporting company's inve...,"Credit reporting, credit repair services, or o..."


In [3]:
# print product type
print(df['Product'].nunique())
print(df['Product'].unique())

# print issue
print(df['Issue'].nunique())
print(df['Issue'].unique())

18
['Credit card or prepaid card'
 'Credit reporting, credit repair services, or other personal consumer reports'
 'Debt collection' 'Vehicle loan or lease' 'Checking or savings account'
 'Mortgage' 'Money transfer, virtual currency, or money service'
 'Payday loan, title loan, or personal loan' 'Student loan'
 'Credit reporting' 'Credit card' 'Consumer Loan'
 'Bank account or service' 'Money transfers' 'Payday loan' 'Prepaid card'
 'Other financial service' 'Virtual currency']
157
['Problem when making payments'
 "Problem with a credit reporting company's investigation into an existing problem"
 'Incorrect information on your report' 'Written notification about debt'
 'Struggling to pay your loan' 'Managing an account'
 'Problems at the end of the loan or lease'
 'Problem with a purchase shown on your statement'
 'Improper use of your report' 'Attempts to collect debt not owed'
 'Trouble during payment process' 'Opening an account'
 'Problem with a lender or other company charging you

We expect to have 18 topics

In [4]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 3 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   Consumer complaint narrative  400000 non-null  object
 1   Issue                         400000 non-null  object
 2   Product                       400000 non-null  object
dtypes: object(3)
memory usage: 515.9 MB


In [5]:
data = df['Consumer complaint narrative'].values.tolist()

# Remove senssored words (XX, XXXX, ...)
data = [re.sub('X{2,}', '', word) for word in data]

# Remove newline and spaces characters
data = [re.sub('[\s\n]+', ' ', word) for word in data]

# Remove distracting slash character
data = [re.sub('\/', '', word) for word in data]

In [6]:
import random
pprint(data[10000])
# pprint(data[random.randint(0,400000)])


('This is my 9th attempt to dispute the information in my credit file which '
 'resulted from an alleged identity theft. Again, Im sending over an identity '
 'theft report # . Pursuant of FCRA section 605B ( a ) ( 1 ) ( 2 ) Credit '
 'Bureaus are required to remove any information in my file that resulted from '
 'an alleged identity theft, not later than 4 business days after the date of '
 'receipt with ( 1 ) appropriate proof of the identity of the consumer ; ( 2 ) '
 'a copy of an identity theft report. Please block following items listed. , '
 'CA ; , CA , CA , CA Balance Owed : {$7300.00} Balance Owed : {$7.00}, US SM '
 'BUS ADMIN Inquiry Date  Inquiry  Inquiry Date  Inquiry Date  Inquiry Date  '
 'Inquiry Date  Inquiry Date  Inquiry Date  Inquiry Date  Inquiry Date ')


In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [8]:
print(data_words[90])

['recently', 'reviewed', 'copy', 'of', 'my', 'credit', 'report', 'was', 'shocked', 'to', 'learn', 'that', 'have', 'been', 'victim', 'of', 'identity', 'theft', 'found', 'names', 'on', 'my', 'credit', 'report', 'that', 'are', 'not', 'me', 'this', 'has', 'been', 'hurting', 'me', 'as', 'am', 'in', 'the', 'process', 'of', 'purchasing', 'new', 'home', 'for', 'my', 'family', 'please', 'remove', 'these', 'fraudulent', 'names', 'from', 'my', 'credit', 'report', 'as', 'soon', 'as', 'possible']


In [9]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\chuk
[nltk_data]     bert\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
stop_words = stopwords.words('english')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [11]:
stop_words.extend(['th'])

In [12]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=4, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [13]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

# def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
#     """https://spacy.io/api/annotation"""
#     texts_out = []
#     for sent in texts:
#         doc = nlp(" ".join(sent)) 
#         texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
#     return texts_out

# for doc in nlp.pipe(docs, batch_size=32, n_process=3, disable=["parser", "ner"]):
#     print([tok.lemma_ for tok in doc])
    
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    for doc in nlp.pipe(texts, batch_size=1000, n_process=-1, disable=["parser", "ner"]):
        lemmas = [token.lemma_ for token in doc if token.pos_ in allowed_postags]
        yield lemmas

def list2string(texts):
    for doc in texts:
        result = ' '.join(doc)
        yield result

In [14]:
# Remove Stop Words
data_words = remove_stopwords(data_words)

In [15]:
# Form Bigrams
data_words = make_bigrams(data_words)

In [16]:
# Form Trigrams
data_words = make_trigrams(data_words)

In [17]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm')

In [18]:

# Do lemmatization keeping only noun, adj, vb, adv
# data_words = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

data_words = list(lemmatization(list2string(data_words), allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']))

In [19]:
print(data_words[1])

['identity', 'steal', 'create', 'fraudelunet', 'account', 'name', 'm', 'try', 'buy', 'house', 'notoice', 'bad', 'thing', 'credit', 'get', 'car', 'loan', 'buy', 'house', 'depend', 'collection', 'never', 'hear', 'payment', 'due', 'time', 'person', 'ruin', 'life', 'police', 'report', 'idenity', 'theft']


In [20]:
# Create Dictionary
id2word = corpora.Dictionary(data_words)

In [21]:
# Term Document Frequency
corpus = [id2word.doc2bow(doc) for doc in data_words]

In [22]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('account', 1),
  ('bill', 1),
  ('card', 2),
  ('complaint', 1),
  ('corporate', 1),
  ('correct', 1),
  ('credit', 2),
  ('due', 1),
  ('dupe', 1),
  ('file', 1),
  ('fortiva', 1),
  ('fortiva_retail', 2),
  ('furniture', 1),
  ('go', 2),
  ('however', 1),
  ('minimum', 8),
  ('month', 4),
  ('monthly', 5),
  ('notcould', 1),
  ('office', 1),
  ('open', 1),
  ('pay', 3),
  ('payment', 8),
  ('prior', 1),
  ('purchase', 2),
  ('reduce', 3),
  ('representative', 1),
  ('sale', 1),
  ('say', 2),
  ('statement', 2),
  ('tell', 1),
  ('time', 2)]]

In [23]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=18, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [24]:
# Print the Keyword in the 18 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.081*"call" + 0.054*"tell" + 0.051*"get" + 0.046*"say" + 0.033*"go" + '
  '0.030*"ask" + 0.027*"try" + 0.027*"time" + 0.025*"never" + 0.025*"back"'),
 (1,
  '0.442*"account" + 0.065*"open" + 0.062*"balance" + 0.057*"date" + '
  '0.032*"close" + 0.024*"last" + 0.017*"status" + 0.015*"signature" + '
  '0.011*"activity" + 0.011*"mine"'),
 (2,
  '0.190*"report" + 0.185*"credit" + 0.055*"information" + 0.048*"remove" + '
  '0.033*"inquiry" + 0.029*"dispute" + 0.026*"inaccurate" + 0.023*"reporting" '
  '+ 0.018*"item" + 0.018*"delete"'),
 (3,
  '0.062*"law" + 0.050*"provide" + 0.032*"require" + 0.032*"verify" + '
  '0.028*"violation" + 0.027*"proof" + 0.026*"act" + 0.026*"legal" + '
  '0.026*"fair" + 0.025*"action"'),
 (4,
  '0.103*"identity" + 0.086*"theft" + 0.070*"balance" + 0.044*"block" + '
  '0.044*"item" + 0.043*"fraudulent" + 0.036*"victim" + 0.032*"file" + '
  '0.031*"information" + 0.028*"talk"'),
 (5,
  '0.135*"bank" + 0.113*"check" + 0.069*"money" + 0.053*"fund" + '
  '0

In [25]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.411563903473326


In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
16    -0.245593 -0.184164       1        1  19.591065
1     -0.178551  0.163537       2        1  17.611872
5     -0.251376 -0.123002       3        1  12.935754
4     -0.092143 -0.160060       4        1   7.213121
17     0.002331 -0.037206       5        1   6.539247
13    -0.050664  0.321968       6        1   6.477620
3     -0.112083 -0.012580       7        1   6.415169
11    -0.079697 -0.154319       8        1   4.982068
6     -0.057550  0.274870       9        1   4.013575
0      0.026138  0.040749      10        1   2.526896
14     0.051758  0.061981      11        1   2.434394
9      0.033486  0.117410      12        1   2.375924
10    -0.032537 -0.061383      13        1   2.126863
7      0.115321 -0.014112      14        1   1.547502
15     0.170216 -0.048127      15        1   1.250998
2      0.212590 -0.054202      16        1   1.092108
8      0.239821 -0.068681      17        1   0.663848
12     0.248533 -0.062679      18        1   0.201975, topic_info=            Term          Freq         Total Category  logprob  loglift
0        account  23114.000000  23114.000000  Default  30.0000  30.0000
5         credit  22097.000000  22097.000000  Default  29.0000  29.0000
50        report  18908.000000  18908.000000  Default  28.0000  28.0000
96      consumer  10865.000000  10865.000000  Default  27.0000  27.0000
19       payment   8154.000000   8154.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
1562        scam      0.046139      1.132329  Topic18 -10.4010   3.0044
1655        post      0.046145      1.132802  Topic18 -10.4008   3.0041
1623  transunion      0.046146      1.134152  Topic18 -10.4008   3.0030
2585    coinbase      0.046133      1.132043  Topic18 -10.4011   3.0045
2189     exhibit      0.046134      1.132609  Topic18 -10.4011   3.0040

[879 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
1237      9  0.992469   acceptance
78        6  0.999123   accordance
1064     18  0.964041  accordingly
0         2  0.001990      account
0         5  0.939318      account
...     ...       ...          ...
182       4  0.120808         year
182       8  0.110673         year
402       3  0.754411          yet
402       9  0.002297          yet
402      13  0.242284          yet

[1235 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[17, 2, 6, 5, 18, 14, 4, 12, 7, 1, 15, 10, 11, 8, 16, 3, 9, 13])